##Serving Models with Microsoft Azure ML

This notebook is part of "loan classifier" based on the [loan classifier](/Users/bhavin.kukadia@databricks.com/Workshops/Share-Git/4-Workshop/02-MLflow Primer).
The [MLFlow guide](https://docs.azuredatabricks.net/spark/latest/mllib/mlflow.html), **MLflow Quick Start: Model Training and Logging**, focuses on training a model and logging the training metrics, parameters, and model to the MLflow tracking server. 

##### NOTE: We do not recommend using *Run All* because it takes several minutes to deploy and update models; models cannot be queried until they are active.

This part of the guide consists of the following sections:

#### Setup
* Launch an Azure Databricks cluster
* Install MLflow
* Install the Azure ML SDK
* Create or load an Azure ML Workspace

#### Building an Azure Container Image for model deployment
* Use MLflow to build a Container Image for the trained model

#### Deploying the model to "dev" using Azure Container Instances (ACI)
* Create an ACI webservice deployment using the model's Container Image

#### Querying the deployed model in "dev"
* Load a sample input vector from the dataset
* Evaluate the sample input vector by sending an HTTP request

#### Deploying the model to production using Azure Kubernetes Service (AKS)
* Option 1: Create a new AKS cluster
* Option 2: Connect to an existing AKS cluster
* Deploy to the model's image to the specified AKS cluster

#### Querying the deployed model in production
* Load a sample input vector from the dataset
* Evaluate the sample input vector by sending an HTTP request

#### Updating the production deployment
* Build an Azure Container Image for another model
* Deploy the new model's image to the AKS cluster
* Query the updated model

#### Cleaning up the deployments
* Terminate the "dev" ACI webservice
* Terminate the production AKS webservice
* Remove the AKS cluster from the Azure ML Workspace

As in the first part of the Quick Start tutorial, this notebook uses ElasticNet models trained on the `diabetes` dataset in scikit-learn.

**Note:** This notebook expects that you use an Azure Databricks hosted MLflow tracking server. If you would like to preview the Azure Databricks MLflow tracking server, contact your Azure Databricks sales representative to request access. To set up your own tracking server, see the instructions in [MLflow Tracking Servers](https://www.mlflow.org/docs/latest/tracking.html#mlflow-tracking-servers) and configure your connection to your tracking server by running [mlflow.set_tracking_uri](https://www.mlflow.org/docs/latest/python_api/mlflow.html#mlflow.set_tracking_uri).

## Prerequisites
Trained classifier based on this notebook in [part 1](/Users/bhavin.kukadia@databricks.com/Workshops/Share-Git/4-Workshop/02-MLflow Primer).

## Setup

1. Ensure you are using or create a cluster specifying 
  * **Databricks Runtime Version:** Databricks Runtime 6.2 or above
  * **Python Version:** Python 3
1. Install required libraries or if using Databricks Runtime 6.2 or above (but not Databricks Runtime for ML), run Cmd 6.
   1. Create required libraries.
      * Source **PyPI** and enter `mlflow`.
      * Source **PyPI** and enter `azureml-sdk`.
   1. Install the libraries into the cluster.
1. Attach this notebook to the cluster.

In [6]:
import warnings;
# warnings.filterwarnings("ignore")

dbutils.library.installPyPI("mlflow")
dbutils.library.installPyPI("azureml-mlflow")
dbutils.library.restartPython()

### Before you begin you'll need
- azure subscription details
- tenant_id = aad_tenant_id
- azure subscription_id
- azure databricks workspace_location e.g. "westus2"
- azure service_principal_id
- azure service_principal_secret

In [8]:
%run /Users/bhavin.kukadia@databricks.com/_helper

In [9]:
# azure subscription details
tenant_id = aad_tenant_id
subscription_id = subscription_id
workspace_location="westus2"
service_principal_id = svc_id_test
service_principal_secret = svc_secret_test
# service principal having access to azure ml workspace
print("service_principal_id", service_principal_id)
print("service_principal_secret",service_principal_secret)

# data for prediction/testing
dataset_path = "/FileStore/tables/loan_test_data-3cd24.csv"
dataset_path_old = "/FileStore/tables/loan_stats_test_old-a4d0a.csv"
#azure ml workspace name and region where its deployed
workspace_name = "bk-azureml-ws" # createIfNotExists

#azure respurce group and subscription id to use
resource_group = "bk-azureml-rg"

#azure k8s
aks_cluster_name = "bk-aks-cluster" #need to be between 2-16 chars length

#variables based on service and models
aks_prod_webservice_name = "bk-loan-clf-prod-v1"

aml_docker_image_name="bk-loan-clf"

aml_docker_image_description="loan risk classifier for lending club"

aml_dev_webservice_name="bk-loan-clf-dev"

#mlflow run id and model path, model_path == model name that you have used to save it in mlflow
model_path = "bk-loan-clf"
model_name = "bk-loan-clf"

model_uri = "runs:/1615910c64954a4eb42f8d53516eeb7b/bk-loan-clf" # from previous notebook

# os.environ.get("AZUREML_PASSWORD")
# model_path2 = "/dbfs/databricks/mlflow/461163625925097/680d79ab749b47379c42d83e2c041e1d/artifacts/xgboost"
# model_path3 = "/dbfs/databricks/mlflow/461163625925097/50bc9ac5a16f464584a7cebad398173a/artifacts/xgboost"

service_principal_id [REDACTED]
service_principal_secret [REDACTED]

### Create or load an Azure ML Workspace

Before models can be deployed to Azure ML, you must create or obtain an Azure ML Workspace. The `azureml.core.Workspace.create()` function will load a workspace of a specified name or create one if it does not already exist. For more information about creating an Azure ML Workspace, see the [Azure ML Workspace management documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-workspace).

[Use Service Principal to access Azure ML Workspace](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azure-ml.ipynb)

In [12]:
import azureml
from azureml.core import Workspace
from azureml.core.authentication import ServicePrincipalAuthentication

svc_pr = ServicePrincipalAuthentication(
    tenant_id=tenant_id,
    service_principal_id=service_principal_id,
    service_principal_password=service_principal_secret)

workspace = Workspace.create(name = workspace_name,
                             location = workspace_location,
                             resource_group = resource_group,
                             subscription_id = subscription_id,
                             exist_ok=True,
                             auth=svc_pr)

print("Found workspace {} at location {} with these images {}".format(workspace.name,workspace.location,list(workspace.images.keys())))

# ws = Workspace(
#     subscription_id=subscription_id,
#     resource_group=resource_group,
#     workspace_name=workspace_name,
#     auth=svc_pr
#     )

/local_disk0/pythonVirtualEnvDirs/virtualEnv-dfc1d898-f8d3-4eb3-9f5c-3c60ef7bc666/lib/python3.7/site-packages/azureml/core/workspace.py:856: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 images = Image.list(self)
/local_disk0/pythonVirtualEnvDirs/virtualEnv-dfc1d898-f8d3-4eb3-9f5c-3c60ef7bc666/lib/python3.7/site-packages/azureml/core/image/image.py:815: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 image = cls(None)
Found workspace bk-azureml-ws at location westus2 with these images ['cost-prdctr', 'bk-loan-clf']

## Building an Azure Container Image for model deployment

### Use MLflow to build a Container Image for the trained model

Use the `mlflow.azuereml.build_image` function to build an Azure Container Image for the trained MLflow model. This function also registers the MLflow model with a specified Azure ML workspace. The resulting image can be deployed to Azure Container Instances (ACI) or Azure Kubernetes Service (AKS) for real-time serving.

## Review the experiment

1. Open the experiment `/Shared/experiments/DiabetesModel` in the workspace.
1. Click a date to view a run.

Specify the run ID associated with an ElasticNet training run from [part 1 of the Quick Start guide](https://docs.azuredatabricks.net/spark/latest/mllib/mlflow-tracking.html). You can find a run ID and model path from the experiment run, which can be found on the run details page:

![image](https://docs.azuredatabricks.net/_static/images/mlflow/mlflow-deployment-example-run-info.png)

### load model from mlflow and containerize it on azureml using acs/aks

In [18]:
%run /Users/bhavin.kukadia@databricks.com/Workshops/AllinOne/_helper_azureml

In [19]:
model_uri
# runs:/1615910c64954a4eb42f8d53516eeb7b/bk-loan-clf

Out[ 13 ]: 'models:/bk-loan-clf/production'

In [20]:
experiment_name = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
print(experiment_name)
experiment_id = client.get_experiment_by_name("02 MLflow Primer").experiment_id

print("experiment_id:",experiment_id)
print("experiment_name:", experiment_name)

runs = client.list_run_infos(experiment_id)
print("#runs:",len(runs))
for info in runs:
    print(info.run_id)
    run_id = info.run_id  

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-4392476077484114> in <module> () 
 1 experiment_name = dbutils . notebook . entry_point . getDbutils ( ) . notebook ( ) . getContext ( ) . notebookPath ( ) . get ( ) 
 2 print ( experiment_name ) 
 ----> 3 experiment_id = client . get_experiment_by_name ( "02 MLflow Primer" ) . experiment_id 
 4 
 5 print ( "experiment_id:" , experiment_id ) 

 AttributeError : 'NoneType' object has no attribute 'experiment_id'

In [21]:
%fs ls /databricks/mlflow/2534614884196308

In [22]:
%fs ls /databricks/mlflow/2534614884196308/02f1e58a9f244281b7984c63535e5d90/artifacts/bk-loan-clf/sparkml/

In [23]:
import mlflow.azureml

model_uri = "runs:/1615910c64954a4eb42f8d53516eeb7b/bk-loan-clf"

# mlflow.azureml.build_image(model_uri, workspace, image_name=None, model_name=None, mlflow_home=None, description=None, tags=None, synchronous=True)
model_image, azure_model = mlflow.azureml.build_image(model_uri=model_uri, 
                                                      workspace=workspace,
                                                      model_name=model_name,
                                                      image_name=aml_docker_image_name,
                                                      description=aml_docker_image_description,
                                                      synchronous=False)

Registering model bk-loan-clf
/local_disk0/pythonVirtualEnvDirs/virtualEnv-dfc1d898-f8d3-4eb3-9f5c-3c60ef7bc666/lib/python3.7/site-packages/mlflow/azureml/__init__.py:197: DeprecationWarning: ContainerImage class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 tags=tags,
/local_disk0/pythonVirtualEnvDirs/virtualEnv-dfc1d898-f8d3-4eb3-9f5c-3c60ef7bc666/lib/python3.7/site-packages/azureml/core/image/container.py:159: DeprecationWarning: ContainerImageConfig class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 base_image, base_image_registry, cuda_version=cuda_version)
/local_disk0/pythonVirtualEnvDirs/virtualEnv-dfc1d898-f8d3-4eb3-9f5c-3c60ef7bc666/lib/python3.7/site-packages/mlflow/azureml/__init__.py:202: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 models=[registered_model])
Creating image
/local_disk0/pythonVirtualEnvDirs/virtualEnv-dfc1d898-f8d3-4eb3-9f5c-3c60ef7bc666/lib/python3.7/site-packages/azureml/core/image/image.py:401: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 image = Image(workspace, id=image_id)

In [24]:
model_image.wait_for_creation(show_output=True)

--------------------------------------------------------------------------- 
 WebserviceException Traceback (most recent call last)
 <command-4392476077484118> in <module> 
 ----> 1 model_image . wait_for_creation ( show_output = True ) 

 /local_disk0/pythonVirtualEnvDirs/virtualEnv-dfc1d898-f8d3-4eb3-9f5c-3c60ef7bc666/lib/python3.7/site-packages/azureml/core/image/image.py in wait_for_creation (self, show_output) 
 446 'current state: {}\n' 
 447 'Error response from server:\n' 
 --> 448 '{}'.format(self.creation_state, error_response), logger=module_logger)
 449 
 450 print ( 'Image creation operation finished for image {}, operation "{}"' . format ( self . id , operation_state ) ) 

 WebserviceException : WebserviceException:
	Message: Image creation polling reached non-successful terminal state, current state: Failed
Error response from server:
StatusCode: 400
Message: Docker image build failed.
	InnerException None
	ErrorResponse 
{
 "error": {
 "message": "Image creation polling reached non-successful terminal state, current state: Failed\nError response from server:\nStatusCode: 400\nMessage: Docker image build failed."
 }
}

In [25]:
model_image

## Deploying the model to "dev" using [Azure Container Instances (ACI)](https://docs.microsoft.com/en-us/azure/container-instances/)

The [ACI platform](https://docs.microsoft.com/en-us/azure/container-instances/) is the recommended environment for staging and developmental model deployments.

### Create an ACI webservice deployment using the model's Container Image

Using the Azure ML SDK, deploy the Container Image for the trained MLflow model to ACI.

In [28]:
from azureml.core.webservice import AciWebservice, Webservice

dev_webservice_name = aml_dev_webservice_name
dev_webservice_deployment_config = AciWebservice.deploy_configuration()
dev_webservice = Webservice.deploy_from_image(name=dev_webservice_name, image=model_image, deployment_config=dev_webservice_deployment_config, workspace=workspace)

In [29]:
dev_webservice.wait_for_deployment()

## Querying the deployed model in "dev"

### Load diabetes dataset

In [32]:
from numpy import *
import pandas as pd
import numpy as np

# load data
dataset = pd.read_csv(dataset_path, delimiter=",")
dataset

## Create sample input vector

In [34]:
import pandas as pd
import numpy as np

dataset = dataset.dropna(axis=0)
feature_cols = ['pregnancies', 'plasma glucose', 'blood pressure','triceps skin thickness', 'insulin', 'bmi', 'diabetes pedigree', 'age']

X = dataset[feature_cols]
y = dataset.diabetes
Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
all_cols = ['pregnancies', 'plasma glucose', 'blood pressure','triceps skin thickness', 'insulin', 'bmi', 'diabetes pedigree', 'age','diabetes']
data = pd.DataFrame(d, columns=all_cols)
sample = data.drop(["diabetes"], axis=1).iloc[[0]]

print(sample)
                                                 
query_input = sample.to_json(orient='split')
query_input = eval(query_input)
query_input.pop('index', None)

#### Evaluate the sample input vector by sending an HTTP request
Query the ACI webservice's scoring endpoint by sending an HTTP POST request that contains the input vector.

In [36]:
import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=json.dumps(inputs), headers=headers)
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

In [37]:
# dev_webservice.scoring_uri
scoring_uri = "http://52.188.221.232:80/score"

In [38]:
query_input2 = {'columns': ['pregnancies', 'plasma glucose', 'blood pressure', 'triceps skin thickness', 'insulin', 'bmi', 'diabetes pedigree', 'age'], 'data': [[6.0, 50.0, 72.0, 35.0, 0.0, 40.0, 0.627, 50.0]]}

In [39]:
#dev_prediction = query_endpoint_example(scoring_uri=dev_webservice.scoring_uri, inputs=query_input)

dev_prediction = query_endpoint_example(scoring_uri=scoring_uri, inputs=query_input)

## Deploying the model to production using [Azure Kubernetes Service (AKS)](https://azure.microsoft.com/en-us/services/kubernetes-service/). Do Option 1 or Option 2.

### Option 1: Create a new AKS cluster

If you do not have an active AKS cluster for model deployment, create one using the Azure ML SDK.

In [42]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this)
prov_config = AksCompute.provisioning_configuration()
aks_cluster_name = aks_cluster_name
# Create the cluster
aks_target = ComputeTarget.create(workspace = workspace, 
                                  name = aks_cluster_name, 
                                  provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

### Option 2: Connect to an existing AKS cluster

If you already have an active AKS cluster running, you can add it to your Workspace using the Azure ML SDK.

In [44]:
# from azureml.core.compute import AksCompute, ComputeTarget

# # Get the resource id from https://porta..azure.com -> Find your resource group -> click on the Kubernetes service -> Properties
# resource_id = "/subscriptions/<subscription-id>/resourcegroups/<resource-group>/providers/Microsoft.ContainerService/managedClusters/<aks-service-name>"

# # Give the cluster a local name
# cluster_name = "<cluster-name>"

# # Attatch the cluster to your workgroup
# aks_target = AksCompute.attach(workspace=workspace, name=cluster_name, resource_id=resource_id)

# # Wait for the operation to complete
# aks_target.wait_for_completion(True)
# print(aks_target.provisioning_state)
# print(aks_target.provisioning_errors)

### Deploy to the model's image to the specified AKS cluster

In [46]:
from azureml.core.webservice import Webservice, AksWebservice

# Set configuration and service name
prod_webservice_name = aks_prod_webservice_name
prod_webservice_deployment_config = AksWebservice.deploy_configuration()

# Deploy from image
prod_webservice = Webservice.deploy_from_image(workspace = workspace, 
                                               name = prod_webservice_name,
                                               image = model_image,
                                               deployment_config = prod_webservice_deployment_config,
                                               deployment_target = aks_target)

In [47]:
# Wait for the deployment to complete
prod_webservice.wait_for_deployment(show_output = True)

## Querying the deployed model in production

#### Evaluate the sample input vector by sending an HTTP request
Query the AKS webservice's scoring endpoint by sending an HTTP POST request that includes the input vector. The production AKS deployment may require an authorization token (service key) for queries. Include this key in the HTTP request header.

In [50]:
import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=json.dumps(inputs), headers=headers)
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

In [51]:
prod_scoring_uri = prod_webservice.scoring_uri
prod_service_key = prod_webservice.get_keys()[0] if len(prod_webservice.get_keys()) > 0 else None

In [52]:
prod_prediction1 = query_endpoint_example(scoring_uri=prod_scoring_uri, service_key=prod_service_key, inputs=query_input)

## Updating the production deployment

### Build an Azure Container Image for the new model

In [55]:
run_id2=run_id2

In [56]:
print(model_name+"-updated")

In [57]:
import mlflow.azureml

model_image_updated, azure_model_updated = mlflow.azureml.build_image(model_path=model_path, 
                                                                      workspace=workspace, 
                                                                      run_id=run_id2,
                                                                      model_name=model_name+"-updated",
                                                                      image_name=aml_docker_image_name+"-updated",
                                                                      description=aml_docker_image_description,
                                                                      synchronous=False)

In [58]:
model_image_updated.wait_for_creation(show_output=True)

### Deploy the new model's image to the AKS cluster

Using the [`azureml.core.webservice.AksWebservice.update()`](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.akswebservice?view=azure-ml-py#update) function, replace the deployment's existing model image with the new model image.

In [60]:
prod_webservice.update(image=model_image_updated)

In [61]:
prod_webservice.wait_for_deployment(show_output = True)

### Query the updated model

In [63]:
prod_prediction2 = query_endpoint_example(scoring_uri=prod_scoring_uri, service_key=prod_service_key, inputs=query_input2)

## Compare the predictions

In [65]:
print("Run ID: {} Prediction: {}".format(run_id1, prod_prediction1)) 
print("Run ID: {} Prediction: {}".format(run_id2, prod_prediction2))

## Cleaning up the deployments

### Terminate the "dev" ACI webservice

Because ACI manages compute resources on your behalf, deleting the "dev" ACI webservice will remove all resources associated with the "dev" model deployment

In [68]:
dev_webservice.delete()

### Terminate the production AKS webservice

This terminates the real-time serving webservice running on the specified AKS cluster. It **does not** terminate the AKS cluster.

In [70]:
prod_webservice.delete()

### Remove the AKS cluster from the Azure ML Workspace

If the cluster was created using the Azure ML SDK (see **Option 1: Create a new AKS cluster**), remove it from the Azure ML Workspace will terminate the cluster, including all of its compute resources and deployments.

If the cluster was created independently (see **Option 2: Connect to an existing AKS cluster**), it will remain active after removal from the Azure ML Workspace.

In [72]:
aks_target.delete()